In [ ]:
!pip install sec_edgar_downloader

     |████████████████████████████████| 1.2 MB 8.2 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires typing-extensions~=3.7.4, but you have typing-extensions 3.10.0.2 which is incompatible.


In [ ]:
!pip install pandas_read_xml

     |████████████████████████████████| 138 kB 9.0 MB/s 
     |████████████████████████████████| 496 kB 59.1 MB/s 
     |████████████████████████████████| 62 kB 892 kB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from sec_edgar_downloader import Downloader
import pandas as pd
import pandas_read_xml as pdx
import xml.etree.ElementTree as ET
import numpy as np
from lxml import etree
from bs4 import BeautifulSoup
import glob
from random import randint
import time
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: ignored

In [ ]:
dl = Downloader('/content/drive/My Drive/Analytics in Practice APG/Data')

In [ ]:
scrap_list = pd.read_csv("/content/drive/My Drive/Analytics in Practice APG/Data/manager_list.csv")
scrap_list = list(scrap_list['CIK'])
scrap_list

In [ ]:
filings = ["13F-HR"]
for equity_id in scrap_list:
    for i in filings:
      try:
        dl.get(i, equity_id, amount=1)
        print(equity_id)
      except:
        time.sleep(20)
        print('Error {}'.format(equity_id))

In [ ]:
text_list = glob.glob("/content/drive/My Drive/Analytics in Practice APG/Data/sec-edgar-filings/*/13F-HR/*/full-submission.txt")
#text_list = glob.glob("/content/drive/My Drive/Analytics in Practice APG/Data/sec-edgar-filings/TWO SIGMA SECURITIES, LLC/13F-HR/*/full-submission.txt")
len(text_list)

In [ ]:
for files in text_list:
  try:
    open_file = open("{}".format(files),"r").read()
    soup = BeautifulSoup(open_file, 'xml')
    result = soup.find_all("XML")
    general_table = result[0]
    investment_table = result[1]
    removal_list = ['<XML>','</XML>']
    for word in removal_list:
      investment_table = str(investment_table).replace(word, "")
    parser = etree.XMLParser(recover=True)
    root = ET.fromstring(investment_table,parser)
    tree = ET.ElementTree(root)
    file_name = files[73:-48]
    tree.write('/content/drive/My Drive/Analytics in Practice APG/Data/XML files/{}.xml'.format(file_name), encoding='utf-8', xml_declaration=True)
    df = pdx.read_xml('/content/drive/My Drive/Analytics in Practice APG/Data/XML files/{}.xml'.format((file_name)), ['ns0:informationTable'])
    df = pdx.fully_flatten(df)
    df = df.rename(columns={'ns0:infoTable|ns0:nameOfIssuer':'NAME OF ISSUER',
                            'ns0:infoTable|ns0:titleOfClass':'TITLE OF CLASS',
                            'ns0:infoTable|ns0:cusip':'CUSIP',
                            'ns0:infoTable|ns0:value':'VALUE (x$1000)',
                            'ns0:infoTable|ns0:shrsOrPrnAmt|ns0:sshPrnamt':'SHRS OR PRN AMT',
                            'ns0:infoTable|ns0:shrsOrPrnAmt|ns0:sshPrnamtType':'SH/PRN',
                            'ns0:infoTable|ns0:investmentDiscretion':'INVESTMENT DISCRETION',
                            'ns0:infoTable|ns0:otherManager':'OTHER MANAGER',
                            'ns0:infoTable|ns0:putCall':'PUT/CALL',
                            'ns0:infoTable|ns0:votingAuthority|ns0:Sole':'VOTING AUTHORITY SOLE',
                            'ns0:infoTable|ns0:votingAuthority|ns0:Shared':'VOTING AUTHORITY SHARED',
                            'ns0:infoTable|ns0:votingAuthority|ns0:None':'VOTING AUTHORITY NONE'})
    df = df.drop(['@xmlns:ns0', '@xmlns:xsi', '@xsi:schemaLocation'], axis=1, errors='ignore')
    df['NEW_CUSIP'] = '#' + df['CUSIP']
    df.to_csv('/content/drive/My Drive/Analytics in Practice APG/Data/Excel files/{}.csv'.format(file_name), index=False)
    print('GOOD {}'.format(file_name))
  except Exception as e:
    print(e)
    print('Error {}'.format(file_name))
    pass